# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# Several references for this study:
# https://www.kaggle.com/chintan5384/ml-pipeline-preparation
# https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

# import libraries
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# load data from database
engine = create_engine('sqlite:///dataframe.db')

# run a query
df = pd.read_sql_table('dataframe', engine)
X = df['message']
Y = df.drop(columns = ['message', 'genre', 'id', 'original'], axis = 1)

In [6]:
# Data exporation
print (Y.sum())
#print (Y.shape)

# Remove column with all '0's or '1's in the whole dataset
del_cols = [i for i in Y.sum()[Y.sum().isin([0, len(Y)])].index]
Y = Y.drop(columns = del_cols)
print (Y.sum())
#print (Y.shape)

related                   20282
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [10]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-zA-Z #+_]')

def tokenize(text):
    tokens = word_tokenize(text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text

    tokens = [w for w in tokens if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [6]:
tokenize(X.loc[0])

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline_RF = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [9]:
pipeline_RF.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000233B6CCFAE8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def performance(model, X_test, y_test):
    '''
    Generate classification report for the model on test dataset
    
    Input: 
        - Model
        - test dataset (X_test and y_test)
        
    Output: 
        - Classification report generated by sklearn.classification_report
          by different prediction categories 
    '''
    
    y_pred = model.predict(X_test)
    precision_list, recall_list, f1_list = [], [], []
    for i, col in enumerate(y_test.columns):
        print (i, col)
        print(classification_report(y_test[col], y_pred[:, i]))
        precision_list.append(float(classification_report(y_test[col], y_pred[:, i]).strip().split()[-4]))
        recall_list.append(float(classification_report(y_test[col], y_pred[:, i]).strip().split()[-2]))
        f1_list.append(float(classification_report(y_test[col], y_pred[:, i]).strip().split()[-3]))
        
    print ("average precision:", np.mean(precision_list))
    print ("average recall:", np.mean(recall_list))
    print ("average f1_score:", np.mean(precision_list))
  

In [10]:
 performance(pipeline_RF, X_test, y_test)

0 related
             precision    recall  f1-score   support

          0       0.62      0.38      0.48      1266
          1       0.82      0.92      0.87      3938
          2       0.67      0.10      0.17        40

avg / total       0.77      0.79      0.77      5244

1 request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.80      0.42      0.55       895

avg / total       0.88      0.88      0.87      5244

2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

3 aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      3113
          1       0.75      0.58      0.66      2131

avg / total       0.75      0.75      0.74      5244

4 medical_help
             precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


28 floods
             precision    recall  f1-score   support

          0       0.94      1.00      0.97      4833
          1       0.86      0.29      0.44       411

avg / total       0.94      0.94      0.93      5244

29 storm
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      4758
          1       0.76      0.41      0.53       486

avg / total       0.93      0.93      0.92      5244

30 fire
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5191
          1       0.00      0.00      0.00        53

avg / total       0.98      0.99      0.98      5244

31 earthquake
             precision    recall  f1-score   support

          0       0.96      0.99      0.98      4766
          1       0.86      0.63      0.72       478

avg / total       0.95      0.96      0.95      5244

32 cold
             precision    recall  f1-score   support

          0       0.98      1.00     

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline_RF.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f9237e94048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
parameters = {
        'clf__estimator__max_depth': [10, 20, 50, None],
        'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__min_samples_split': [2, 5, 10],
        'clf__estimator__n_estimators': [10, 50, 100],
        }


cv = GridSearchCV(pipeline_RF, param_grid=parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
def build_model():
    pipeline_RF = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'clf__estimator__max_depth': [10, 20, None],
        'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__min_samples_split': [2, 5, 10],
        }
    cv = GridSearchCV(pipeline_RF, param_grid=parameters)
    
    return cv

model_RF = build_model()
model_RF.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000233B6CCFAE8>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [10, 20, None],
                         'clf__estimator__min_samples_leaf': [1, 2, 4],
                         'clf__estimator__min_samples_split': [2, 5, 10]})

In [12]:
performance(model_RF, X_test, y_test)

0 related
              precision    recall  f1-score   support

           0       0.72      0.35      0.47      1266
           1       0.81      0.96      0.88      3938
           2       0.75      0.07      0.14        40

    accuracy                           0.80      5244
   macro avg       0.76      0.46      0.50      5244
weighted avg       0.79      0.80      0.78      5244

1 request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.85      0.45      0.59       895

    accuracy                           0.89      5244
   macro avg       0.87      0.72      0.76      5244
weighted avg       0.89      0.89      0.88      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50   

C:\Users\zhu\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


4 medical_help
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4822
           1       0.73      0.08      0.15       422

    accuracy                           0.92      5244
   macro avg       0.83      0.54      0.55      5244
weighted avg       0.91      0.92      0.89      5244

5 medical_products
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4974
           1       0.83      0.09      0.16       270

    accuracy                           0.95      5244
   macro avg       0.89      0.54      0.57      5244
weighted avg       0.95      0.95      0.93      5244

6 search_and_rescue
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5117
           1       0.63      0.09      0.16       127

    accuracy                           0.98      5244
   macro avg       0.80      0.55      0.58      5244
weighted avg       0

In [15]:
model_RF.best_params_

{'clf__estimator__max_depth': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline_GB = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])
        

pipeline_GB.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000233B6CCFAE8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=GradientBoostingClassifier()))])

In [17]:
performance(pipeline_GB, X_test, y_test)

0 related
              precision    recall  f1-score   support

           0       0.76      0.22      0.34      1266
           1       0.79      0.98      0.87      3938
           2       0.39      0.23      0.29        40

    accuracy                           0.79      5244
   macro avg       0.65      0.47      0.50      5244
weighted avg       0.78      0.79      0.74      5244

1 request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.47      0.60       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50   

In [17]:
pipeline_GB.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f12a0730620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
                 learning_rate=0.1, loss='deviance', max_depth=3,
                 max_features=None, max_leaf_nodes=None,
                 min_impurity_decrease=0.0, min_impurity_split=None,
                 min_samples_leaf=1, min_samples_split=2,
                 min_weight_fraction_leaf=0.0, n_estimators=100,
               

In [23]:
def build_model():
    pipeline_GB = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
        ])
        
    
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [10, 50, 100]
    }

    cv = GridSearchCV(pipeline_GB, param_grid=parameters)
    
    return cv

model_GB = build_model()
model_GB.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000233B6CCFAE8>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=GradientBoostingClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 50, 100],
                         'vect__ngram_range': ((1, 1), (1, 2))})

In [25]:
performance(model_GB, X_test, y_test)

0 related
              precision    recall  f1-score   support

           0       0.76      0.22      0.34      1266
           1       0.79      0.98      0.87      3938
           2       0.47      0.17      0.25        40

    accuracy                           0.79      5244
   macro avg       0.67      0.46      0.49      5244
weighted avg       0.78      0.79      0.74      5244

1 request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.47      0.60       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50   

### 9. Export your model as a pickle file

In [27]:
pickle_out = open("RF_pipeline.pickle", 'wb')
pickle.dump(model_RF, pickle_out)
pickle_out.close()

In [28]:
pickle_out = open("GB_pipeline.pickle", 'wb')
pickle.dump(model_GB, pickle_out)
pickle_out.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.